In [2]:
!pip install pymorphy2
!pip install pyLDAvis
!pip install razdel
import gensim
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")

     |████████████████████████████████| 1.7 MB 1.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=da2f44270172b16d3392d9c9f00a4bd3a817b70a652cce438475b449d1ca1c60
  Stored in directory: /Users/anna/Library/Caches/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=73f21fd96d808f37fee7ecb39e4aa7a276fc3d012a087efe4535cb0b00833099
  Stored in directory: /Users/anna/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built pyLDAvis sklearn


/Users/anna/opt/anaconda3/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Users/anna/opt/anaconda3/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Users/anna/opt/anaconda3/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/Users/anna/opt/anaconda3/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspe

In [4]:
import nltk

In [15]:
# добавим лемматизацию
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

Домашнее задание № 5. Матричные разложения/Тематическое моделирование
Задание № 1 (4 балла)
Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier, RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 10 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.
Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [20]:
data = pd.read_csv('avito_category_classification.csv')

In [21]:
data['description_norm'] = data['description'].apply(normalize)

In [92]:
pipeline_nmf_sgdc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(40)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))
])

In [93]:
pipeline_nmf_kn = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(40)),
    ('clf', KNeighborsClassifier(n_neighbors=3))
])

In [53]:
pipeline_nmf_et = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', ExtraTreesClassifier(n_estimators=100))
])

In [71]:
pipeline_svd_sgdc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(100)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))
])

In [73]:
pipeline_svd_rf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(100)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))
])

In [77]:
pipeline_svd_kn = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(100)),
    ('clf', KNeighborsClassifier(n_neighbors=3))
])

In [75]:
pipeline_svd_et = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(100)),
    ('clf', ExtraTreesClassifier(n_estimators=100))
])

In [58]:
def eval_table(X, y, pipeline, N=6):
    # зафиксируем порядок классов
    labels = list(set(y))
    
    # метрики отдельных фолдов будет хранить в табличке
    fold_metrics = pd.DataFrame(index=labels)
    # дополнительно также соберем таблицу ошибок
    errors = np.zeros((len(labels), len(labels)))
    
    # создаем стратегию кросс-валидации
    # shuffle=True (перемешивание) - часто критично важно указать
    # т.к. данные могут быть упорядочены и модель на этом обучится
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        # fit-predict как и раньше, но сразу пайплайном
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        # записываем метрику и индекс фолда
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    
    # таблица для усредненных значений
    # тут мы берем колонки со значениями и усредняем их
    # часто также все метрики сразу суммируют и в конце просто делят на количество фолдов
    # но мы тут помимо среднего также хотим посмотреть на стандартное отклонение
    # чтобы понять как сильно варьируются оценки моделей
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    # добавим одну колонку со средним по всем классам
    result.loc['mean'] = result.mean().round(2)
    # проценты ошибок просто усредняем
    errors /= N
    
    return result, errors

Теперь куча метрик:

In [94]:
metrics_nmf_sgd, errors_nmf_sgd = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_sgdc)

In [95]:
metrics_nmf_rf, errors_nmf_rf = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_rf)

In [69]:
metrics_nmf_kn, errors_nmf_kn = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_kn)

In [64]:
metrics_nmf_et, errors_nmf_et = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_et)

In [72]:
metrics_svd_sgdc, errors_svd_sgdc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_sgdc)

In [74]:
metrics_svd_rf, errors_svd_rf = eval_table(data['description_norm'], data['category_name'], pipeline_svd_rf)

In [78]:
metrics_svd_kn, errors_svd_kn = eval_table(data['description_norm'], data['category_name'], pipeline_svd_kn)

In [76]:
metrics_svd_et, errors_svd_et = eval_table(data['description_norm'], data['category_name'], pipeline_svd_et)

In [85]:
metrics = pd.DataFrame(columns=metrics_svd_rf.columns)

In [87]:
metrics = metrics.append(metrics_nmf_sgdc.loc['mean'])
metrics = metrics.append(metrics_nmf_rf.loc['mean'])
metrics = metrics.append(metrics_nmf_kn.loc['mean'])
metrics = metrics.append(metrics_nmf_et.loc['mean'])
metrics = metrics.append(metrics_svd_sgdc.loc['mean'])
metrics = metrics.append(metrics_svd_rf.loc['mean'])
metrics = metrics.append(metrics_svd_kn.loc['mean'])
metrics = metrics.append(metrics_svd_et.loc['mean'])

In [88]:
print (metrics)

      precision  precision_std  recall  recall_std    f1  f1_std
mean       0.62           0.18    0.49        0.12  0.47    0.09
mean       0.75           0.12    0.46        0.06  0.48    0.06
mean       0.50           0.04    0.49        0.04  0.48    0.03
mean       0.74           0.04    0.68        0.04  0.70    0.03
mean       0.70           0.08    0.64        0.06  0.66    0.03
mean       0.72           0.08    0.46        0.03  0.49    0.03
mean       0.52           0.04    0.44        0.04  0.45    0.03
mean       0.65           0.06    0.48        0.04  0.52    0.04


Видно, что у NMF-RandomForest и NMF-ExtraTreesClassifier самые лучшие показатели.